In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.io import loadmat
from PIL import Image
import math
import pandas as pd
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering
import seaborn as sns

In [ ]:
joint_name = ['HeadF', 'HeadB', 'HeadL', 'SpineF', 'SpineM', 'SpineL', 
            'Offset1', 'Offset2', 'HipL', 'HipR', 'ElbowL', 'ArmL', 
            'ShoulderL', 'ShoulderR', 'ElbowR', 'ArmR', 'KneeR', 
            'KneeL', 'ShinL', 'ShinR']

joints_idx = [[1, 2], [2, 3], [1, 3], [2, 4], [1, 4], [3, 4], [4, 5], 
            [5, 6], [4, 7], [7, 8], [5, 8], [5, 7], [6, 8], [6, 9], 
            [6, 10], [11, 12], [4, 13], [4, 14], [11, 13], [12, 13], 
            [14, 15], [14, 16], [15, 16], [9, 18], [10, 17], [18, 19], 
            [17, 20]]

def loadMatFile(fileName):
    mat = loadmat('trainTestSplit.mat')[fileName]
    print("loading:", fileName)
    return mat

def plotKnownOrder(matFile, img_numb):
    mat = matFile[img_numb]
    x = mat[0]
    y = mat[1]
    z = mat[2]
    # loading plot
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # adding points, labels and lines
    try:
        ax.scatter(x,y,z, color='#ff5e5e', s =10, marker='x')
    except:
        pass
    addLabels(ax, x, y, z)
    drawLines(ax, x, y, z)
    dist = drawAllLines(ax, x, y, z)

    # Labeling plot
    ax.set_title("Rat positioning")
    ax.set_xlabel("X axis")
    ax.set_ylabel("Y axis")
    ax.set_zlabel("Z axis")
    # plt.show()
    ax = heatMap(dist)
    ax.set_title("Distance Matrix")
    ax.set_xlabel("Distance from POI to other points")
    ax.set_ylabel("Point of interest (POI)")
    # plt.show()
    return dist

def addLabels(ax, x, y, z):
    points_array = []
    # run through all the points to add labels
    for i in range(len(x)):
        try:
            points_array.append([x[i], y[i], z[i]])
            # label = np.around(points_array[i]).astype(int)
            # label = joint_name[i]
            label = ""
            ax.text(x[i], y[i], z[i], label)
            # print(i, ":", points_array[i], joint_name[i])
        except:
            pass
    return points_array

def drawLines(ax,x, y, z):
    # run through all the connections to draw the points
    for i in range(len(joints_idx)):
        try: 
            # Getting both points to draw line
            idx = joints_idx[i]
            x_line = [x[idx[0]-1], x[idx[1]-1]]
            y_line = [y[idx[0]-1], y[idx[1]-1]]
            z_line = [z[idx[0]-1], z[idx[1]-1]]
            z_coord_1 = x[idx[0]-1], y[idx[0]-1], z[idx[0]-1]
            z_coord_2 = x[idx[1]-1], y[idx[1]-1], z[idx[1]-1]

            # Draw lines
            if i < 3: 
                ax.plot(x_line, y_line, z_line, c="#064ea1", linewidth=4)
            elif i < 6:
                ax.plot(x_line, y_line, z_line, c="#64ccd1", linewidth=4)
            else:
                ax.plot(x_line, y_line, z_line, c="#46b8a7", linewidth=4)
        except:
            pass

def drawAllLines(ax,x, y, z):
    joint_len = len(joint_name)
    all_lines = []
    distance = []

    for i in range(joint_len):
        point_dist = []
        for j in range(joint_len):
            point_dist.append(measureDistance(x[i], y[i], z[i], x[j], y[j], z[j]))
            if (not([i,j] in all_lines)):
                x_line = [x[i], x[j]]
                y_line = [y[i], y[j]]
                z_line = [z[i], z[j]]
                ax.plot(x_line, y_line, z_line, color='#b1d8fc', linewidth=0.5)
                all_lines.append([i,j])
                all_lines.append([j,i])
        distance.append(point_dist)
    return distance

def getDistance(mat):
    joint_len = len(joint_name)
    x = mat[0]
    y = mat[1]
    z = mat[2]
    distance = []
    for i in range(joint_len):
        point_dist = []
        for j in range(joint_len):
            point_dist.append(measureDistance(x[i], y[i], z[i], x[j], y[j], z[j]))
        distance.append(point_dist)
    return distance

# def getPointDistance(mat):
#     joint_len = len(joint_name)
#     x = mat[0]
#     y = mat[1]
#     z = mat[2]
#     distance = []
#     for i in range(joint_len):
#         point_dist = []
#         for j in range(joint_len):
#             point_dist.append(measureDistance(x[i], y[i], z[i], x[j], y[j], z[j]))
#         distance.append(point_dist)
#     return distance


def measureDistance(x_1, y_1, z_1, x_2, y_2, z_2):
    x = (x_1 - x_2)
    y = (y_1 - y_2)
    z = (z_1 - z_2)
    return np.sqrt(x**2 + y**2 + z**2)

def heatMap(dist):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.imshow(dist) 
    plt.colorbar()
    plt.grid(False)
    return ax

# def getAngles(matFile, numb):
#     mat = matFile[numb]
#     x = np.array(mat[0])
#     y = np.array(mat[1])
#     z = np.array(mat[2])
#     angles = []
#     for i in range(len(x)):
#         curr_point = []
#         for j in range(len(y)):
#             for k in range(len(z)):
#                 a = [x[i], y[i], z[i]]
#                 b = [x[j], y[j], z[j]]
#                 c = [x[k], y[k], z[k]]
#                 if not ((a == b) or (b == c) or (c == a)):
#                     a = np.array(a)
#                     b = np.array(b)
#                     c = np.array(c)
#                     ba = a - b
#                     bc = c - b
#                     cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
#                     angle = np.arccos(cosine)
#                     angle = np.degrees(angle)
#                     curr_point.append(angle)
#                 else:
#                     curr_point.append(0)
#         angles.append(curr_point)
#     angles = np.array(angles)
#     angles = angles.reshape(20, 20, 20)
#     print(angles.shape)
#     return(angles)

def getDistData (fileName, numb):
    mat = loadMatFile(fileName)
    training = []
    for i in range(numb):
        dist = np.array(getDistance(mat[i]))
        for i in dist:
            temp = []
            norm = distNorm(i)
            temp.append(norm)
            training.append(temp)
    training = np.array(training)
    return training

def getLabelData(numb):
    output = []
    for i in range(numb):
        for j in range(len(joint_name)):
            output.append(j)
    output = np.array(output)
    return output

def distNorm(point):
    norm = [float(i)/max(point) for i in point]
    return norm

# def distHist(dist):
#     import matplotlib.pyplot as plt
#     fig = plt.figure()
#     ax = fig.add_subplot(111)
#     ax.set_title("Distance Histogram")
#     ax.set_xlabel("Normalized distances")
#     ax.set_ylabel("Number of distances that fall into bin")
    
#     ax = fig.add_axes([0,0,1,1])
#     labels = []
#     for i in range(len(dist)):
#         labels.append(i)
#     ax.bar(labels,dist)
#     plt.show()

# def distPlot(dist):
#     for i in range(len(dist)-1):
#         plt.plot([i,i+1], [dist[i],dist[i+1]], 'ro-')
#     plt.show()

In [ ]:
def getRelAngles(matFile, numb):
    mat = matFile[numb]
    x = np.array(mat[0])
    y = np.array(mat[1])
    z = np.array(mat[2])
    ba = np.array([0.0, 0.0, 1.0])
    angles = []

    for i in range(len(x)):
        b = np.array([x[i], y[i], z[i]])
        temp = []
        for j in range(len(y)):
            c = np.array([x[j], y[j], z[j]])
            if not np.array_equal(b, c):
                bc = c - b
                cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
                angle = np.arccos(cosine)
                angle = np.degrees(angle)
                temp.append(angle)
            else:
                temp.append(0)
        angles.append(temp)
    ax = heatMap(angles)
    ax.set_title("Angles Matrix")
    ax.set_xlabel("Angle from <0,0,1> to POI")
    ax.set_ylabel("Point of interest (POI)")
    # plt.show
    return angles

In [ ]:
def distBin(dist, numb):
    # indicies = []
    # output = []
    # bins = []
    # for i in range(6):
    #     bins.append(i*0.2)
    # for i in range(len(dist)):
    #     norm_dist = distNorm(dist[i][0])
    #     temp = np.ndarray.tolist(np.digitize(norm_dist, bins))
    #     indicies.append(temp)
    for i in range(numb):
        # distHist(count(indicies[i], bins))
        # distPlot(count(indicies[i], bins))
        # plotHist(count(indicies[i]), bins)
        plotHist(dist)

# def count(indicies):
#     output = []
#     for i in range(6):
#         output.append(indicies.count(i+1))
#     return output

In [ ]:
def clusterPlot(data):
    cluster = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='ward')
    cluster.fit_predict(data)
    
    fig, ax = plt.subplots()
    for i in range(5):
        indices = [j for j, x in enumerate(cluster.labels_) if x == i]
        if len(indices)>2:
            temp_x = []
            temp_y = []
            for j in indices:
                temp_x.append(data[:,0][j])
                temp_y.append(data[:,1][j])
            sns.kdeplot(x = np.array(temp_x), y = np.array(temp_y), shade=True, alpha = .8, cmap='Greys', levels=8)
            
    plt.scatter(data[:,0], data[:,1], c='black', s=100)
    plt.show()

In [ ]:
mat = loadMatFile('split1True')

In [ ]:
for i in range (1):
    dist = plotKnownOrder(mat, i)  
    angles = getRelAngles(mat, i)
    for i in range(len(dist)):
        data = []
        for j in range(len(dist[i])):
            temp = [dist[i][j], angles[i][j]]
            data.append(temp)
        clusterPlot(np.array(data))


In [ ]:
def plotHist(dist):
    for i in range(len(dist)):
        fig, ax = plt.subplots()
        sns_plot = sns.kdeplot(ax = ax, x=dist[i][0], bw_adjust=.5, fill=True, color='black')
        fig.canvas.draw()
        # sns.displot(x=dist[i][0], kde = True, bins=5, color='black')
        ax.set_title("Distance Histogram")
        ax.set_xlabel("Normalized Bins")
        plt.figure()
        canvas = fig.canvas
        grayscale(canvas)

def grayscale(canvas):
        data = np.array(np.frombuffer(canvas.tostring_rgb(), dtype=np.uint8))
        data = data.reshape(canvas.get_width_height()[::-1] + (3,))
        data = np.array(data, dtype=np.uint8)
        img = Image.fromarray(data).convert('L')
        
        # print(data.shape)
        rgb_weights = [0.2989, 0.5870, 0.1140]

        grayscale_image = np.dot(data[...,:3], rgb_weights)
        plt.imshow(grayscale_image, cmap=plt.get_cmap("gray"))
        # print(grayscale_image.shape)
        return grayscale_image

In [ ]:
numb = 1
training_dist = getDistData('split1True', numb)

In [ ]:
distBin(training_dist, numb)

In [ ]:
import tensorflow as tf
numb = 500
training_dist = getDistData('split1True', numb)
testing_dist = getDistData('split2True', numb)
training_label = getLabelData(numb)
testing_label = getLabelData(numb)

In [ ]:
import matplotlib.pyplot as plt
image_index =  199
print(training_label[image_index]) # The label is 8
plt.imshow(testing_dist[image_index])

In [ ]:
training_dist = training_dist.reshape(training_dist.shape[0], 1, 20, 1)
testing_dist = testing_dist.reshape(testing_dist.shape[0], 1, 20, 1)
input_shape = (1, 20, 1)

training_dist = training_dist.astype('float32')
testing_dist = testing_dist.astype('float32')


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, MaxPool2D
input_shape = (20, 1, 1)
model = Sequential()
Conv2D(64, 5, 1, padding="same", activation="relu")
model.add(Dropout(rate=0.2))
model.add(Flatten())
model.add(Dense(64, activation = "relu"))
model.add(Dense(20, activation = "softmax"))

In [ ]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
print(training_dist.shape)
print(training_label.shape)
model.fit(x=training_dist,y=training_label, epochs=50)

In [ ]:
output = model.evaluate(testing_dist, testing_label)
print("Loss:", output[0])
print("Accuracy:", output[1])

In [ ]:
image_index = 233
plt.imshow(testing_dist[image_index])
pred = model.predict(testing_dist[image_index].reshape(1, 1, 20, 1))
print("predict", pred.argmax())
print("actual", testing_label[image_index])

In [ ]:
def plotUnknownOrder(matFile, img_numb):
    mat = matFile[img_numb]
    x = mat[0]
    y = mat[1]
    z = mat[2]

    # loading plot
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # adding points, labels and lines
    try:
        ax.scatter(x,y,z, color='#ff5e5e', s =10, marker='x')
    except:
        pass
    addLabels(ax, x, y, z)
    drawLines(ax, x, y, z)
    dist = drawAllLines(ax, x, y, z)

    # Labeling plot
    ax.set_title("Rat positioning")
    ax.set_xlabel("X axis")
    ax.set_ylabel("Y axis")
    ax.set_zlabel("Z axis")
    plt.show()
    return dist

def order(data):
    output = []
    for i in range(len(data)):
        pred = model.predict(data[i].reshape(1, 1, 20, 1))
        print(pred.argmax())
        output.append(pred.argmax())
    return np.array(output)

In [ ]:
# mat = loadMatFile('split1True')
img_numb = 10

data = getDistData('split1True', img_numb)
data = data.reshape(data.shape[0], 1, 20, 1)
data = data.astype('float32')
temp = order(data)
print(temp)
# for i in range (1):
#     dist = printPlot(mat, i)  
#     heatMap(dist)